In [6]:
from scipy import signal

import soundfile as sf
import numpy as np
import pathlib
import os
import librosa

In [4]:
test = np.load('./data/1272-128104-0000.npy')

test.shape

(584, 161)

In [ ]:
ROOT = './data'
folder = pathlib.Path(ROOT)
i = 0
for file in folder.glob('*/'):
    if str(file)[-4:] == '.npy':
        test = np.load(os.path.join(ROOT, file.name))

        print(test.shape)
        
        if i == 1000:
            break
            
        i += 1

In [37]:
def log_linear_specgram(audio, sample_rate, window_size=20,
                        step_size=10, eps=1e-10):

    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))

    _, _, spec = signal.spectrogram(audio, fs=sample_rate,
                                    window='hann', nperseg=nperseg, noverlap=noverlap,
                                    detrend=False)

    return np.log(spec.T.astype(np.float32) + eps)

def _normalize_audio_feature(audio_feature):
    """Perform mean and variance normalization on the spectrogram feature.
    Args:
        audio_feature: a numpy array for the spectrogram feature.
    Returns:
        a numpy array of the normalized spectrogram.
    """
    mean = np.mean(audio_feature, axis=0)
    var = np.var(audio_feature, axis=0)
    normalized = (audio_feature - mean) / (np.sqrt(var) + 1e-6)

    return normalized

In [32]:
audio, sr = sf.read('./dev-clean/dev-clean/1272/128104/1272-128104-0000.flac')
print(audio)
print(sr)
len(audio) / sr

[0.00238037 0.0020752  0.00198364 ... 0.00042725 0.00057983 0.0010376 ]
16000


5.855

In [35]:
spec_noah = log_linear_specgram(audio, sr, window_size=20, step_size=10)
print(spec_noah)
print(spec_noah.shape)

[[-22.532352 -20.430473 -20.791428 ... -23.025547 -23.021275 -23.024914]
 [-22.13534  -20.27576  -20.645267 ... -23.021513 -23.024462 -23.0258  ]
 [-23.015457 -19.67066  -20.491922 ... -23.024092 -23.02292  -23.019823]
 ...
 [-22.430746 -19.656033 -20.11614  ... -23.020298 -23.02459  -23.025414]
 [-22.930197 -20.477222 -21.099878 ... -23.025618 -23.023844 -23.02309 ]
 [-21.72683  -20.230967 -21.06793  ... -23.02299  -23.024109 -23.025787]]
(584, 161)


In [42]:
spec_noah = _normalize_audio_feature(spec_noah)

print(spec_noah)

[[-1.5193279  -1.6876487  -1.5093766  ... -0.8995468  -0.89884335
  -0.77739936]
 [-1.3558683  -1.6331824  -1.4732482  ... -0.89792866 -0.9000996
  -0.77779764]
 [-1.7182337  -1.4201583  -1.435344   ... -0.89896303 -0.89949214
  -0.7751136 ]
 ...
 [-1.4774941  -1.4150087  -1.342457   ... -0.8974413  -0.90014994
  -0.7776238 ]
 [-1.6831299  -1.7041065  -1.5856204  ... -0.8995751  -0.89985603
  -0.77658063]
 [-1.1876749  -1.6174128  -1.5777234  ... -0.8985208  -0.8999605
  -0.7777916 ]]


In [44]:
np.expand_dims(spec_noah, axis=2).shape

(584, 161, 1)

In [13]:
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name) 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

path = "../data/toy_raw/84/121550/84-121550-0034.flac"
file = '../data/toy_raw/84/121123/84-121123-0000.flac'

feature = extract_features(path)

In [1]:
import re

def create_character_mapping():

    character_map = {' ': 0}

    for i in range(97, 123):
        character_map[chr(i)] = len(character_map)

    return character_map

char_idx = create_character_mapping()

string = "THIS EVERY OTHER SAVOUR DOTH TRANSCEND AND NOTWITHSTANDING SLAKED SO FAR MAY BE THY THIRST THAT I REVEAL TO THEE NO MORE 12/*1@#$$%"


# print([char_idx[char] for char in string.lower()])

string_aft = ' '.join(re.split('[^a-z]', string.lower()))
print(string_aft)

this every other savour doth transcend and notwithstanding slaked so far may be thy thirst that i reveal to thee no more           
